In [1]:
# GPU 할당확인 
import torch

print('CUDA GPU availalbe : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')

CUDA GPU availalbe : True
1 GPU(s) is(are) allocated


In [2]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import cv2
import copy

import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, lr_scheduler
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter
import time

In [3]:
# import wandb



# config = {"epochs":epochs, "batch_size":batch_size, "learning_rate":lr}
# wandb.init(project="boost_2nd_pstage_1",config=config)

In [4]:
batch_size = 32
epochs = 15
lr = 3e-4
model_name = "argument_control"
seed = 42
val_ratio = 0.2
check_period = 40
fullTrain = False

csv_dir = '/tf/pstage_1/train/train.csv'
train_dir = '/tf/pstage_1/train'
root_dir = '/tf/pstage_1/train/images/'

In [5]:
class MyDataset(Dataset):
    def __init__(self, csv_file, root_dir, is_Train=True, val_ratio = 0.2, seed = 42, full_Train = False, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로
            img_dir (string): 모든 이미지가 존재하는 디렉토리 경로
            transform (callable, optional): 샘플에 적용될 Optional transform
        """
        super().__init__()

        csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        self.img_path = []

        if not full_Train:
            path = csv.path.values
            np.random.seed(seed) 
            np.random.shuffle(path)
            n_val = int(len(path) * val_ratio)
            
            if is_Train:
                people = path[n_val:]
            else:
                people = path[:n_val]
        else:
            people = csv.path.values

        for person in people:
            images = [root_dir + person + '/' + image for image in os.listdir(root_dir + person) if not image[:1] == '.']
            for image in images:
                self.img_path.append(image)

        self.comb_dic = {}
        comb = [(m, g, a) for m in ['m', 'i', 'n'] for g in ['male', 'female'] for a in [0, 1, 2]]
        for i, (m, g, a) in enumerate(comb):
            self.comb_dic[(m, g, a)] = i

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        img_name = self.img_path[idx]

        mask = img_name.split('/')[-1][:1]
        features = img_name.split('/')[-2].split('_')
        gender = features[1]
        if features[3] in ['58', '59']:
            age = 2
        else:
            age = int(features[3]) // 30
        target = self.comb_dic[(mask, gender, age)]
            
        # Albumentations
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:

            img = self.transform(image=img)["image"]
        # Albumentations
        
        return img, target

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
net = Net()

Loaded pretrained weights for efficientnet-b4


In [8]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataset import random_split
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_train = A.Compose([
    A.RandomCrop(height = 450, width = 360),
    A.ShiftScaleRotate(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
#     A.CenterCrop(height=450, width=360),
    A.SmallestMaxSize(max_size=160),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

transform_val = A.Compose([
#     A.ShiftScaleRotate(p=0.5),
    A.CenterCrop(height=450, width=360),
    A.SmallestMaxSize(max_size=160),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Validation set generate
if not fullTrain:
    train_dataset = MyDataset(csv_dir, root_dir, is_Train=True, val_ratio = val_ratio, seed = seed, 
                             full_Train = fullTrain, transform=transform_train)
    val_dataset = MyDataset(csv_dir, root_dir, is_Train=False, val_ratio = val_ratio, seed = seed, 
                             full_Train = fullTrain, transform=transform_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True, num_workers=4)

# All training
else:
    dataset = MyDataset(csv_dir, root_dir, is_Train=True, full_Train = fullTrain, transform=transform_train)
    train_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, num_workers=1, shuffle=True)

In [9]:
# idx = int(random.random()*len(train_dataset))
# # idx = 1637
# print(idx)

# plt.imshow(train_dataset[idx][0])

In [10]:
from torch.optim import AdamW

In [11]:
# -- Focal Loss
# https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/8
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=0.8, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [12]:
#criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = AdamW(net.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.2)

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
net.cuda()
# best_acc = 0.0
os.makedirs('model', exist_ok=True)
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    print('='*12)
    # Train
    bestResult = 0.0
    for iter, (img, label) in enumerate(train_loader):
        start = time.time()
        
        # forward propagation
        optimizer.zero_grad()
        img, label = img.type(torch.FloatTensor).cuda(), label.cuda()
        pred_logit = net(img)
        loss = criterion(pred_logit, label)
        
        # backward propagation
        loss.backward()
        optimizer.step()
        train_loss = loss.item()
        
        end = time.time()
        
        # Check
        if (iter % check_period == 0) or (iter == len(train_loader)-1):
            
            # Validation            
            if not fullTrain:
                valid_loss, valid_acc = AverageMeter(), AverageMeter()

                for img, label in val_loader:
                    img, label = img.type(torch.FloatTensor).cuda(), label.cuda()
                    with torch.no_grad():
                        pred_logit = net(img)
                    loss = criterion(pred_logit, label)

                    # Accuracy 계산
                    pred_label = torch.max(pred_logit, 1)[1]
                    acc = (pred_label == label).sum() / np.float(len(img))

                    valid_loss.update(loss.item(), len(img))
                    valid_acc.update(acc, len(img))

                valid_loss = valid_loss.avg
                valid_acc = valid_acc.avg

                print("Iter [%3d/%3d] | Train Loss %.4f | Valid Loss %.4f | Valid Acc %.4f | Time(iter) %.4f"  %
                    (iter, len(train_loader), train_loss, valid_loss, valid_acc, end - start))
                if valid_acc > bestResult:
                    bestResult = valid_acc
                    best_model_weights = copy.deepcopy(net.state_dict())
                    
            # Full data training check    
            else:
                if train_loss < bestResult:
                    bestResult = train_loss
                    best_model_weights = copy.deepcopy(net.state_dict())
                    
                print("Iter [%3d/%3d] | Train Loss %.4f | Time(iter) %.4f" %
                    (iter, len(train_loader), train_loss, end-start))
        # Check
    # Train
    if epoch < epochs:
        scheduler.step()
        
    torch.save(best_model_weights, f'model/efficientnet_{model_name}_epoch{epoch + 1}.pth')
    print("BEST in epoch {} | Acc {:.4f}".format(epoch + 1, bestResult))
    
print('Finished training')

Epoch 1/15
Iter [  0/473] | Train Loss 2.8100 | Valid Loss 2.7869 | Valid Acc 0.0537 | Time(iter) 0.3265


# Inference

In [ ]:
test_dir = '/tf/pstage_1/eval'

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
#         image = Image.open(self.img_paths[index])
        
        # Albumentations
        image = cv2.imread(self.img_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)["image"]
        # Albumentations
        
        # Ori
#         if self.transform:
#             image = self.transform(image)
        # Ori
        return image

        
    def __len__(self):
        return len(self.img_paths)

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

dataset = TestDataset(image_paths, transform_val)

loader = DataLoader(
    dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model = Net()
model.load_state_dict(torch.load('model/efficientnet_argument_control_epoch1.pth'))
model.eval()
model.cuda()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for img in loader:
    img = img.type(torch.FloatTensor).cuda()
    with torch.no_grad():
        pred_logit = model(img)
        pred = pred_logit.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'testest.csv'), index=False)
print('test inference is done!')